In [2]:
import cv2
import time
import json
import random

In [17]:
cap = cv2.VideoCapture(0)

In [32]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [33]:
while True:
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #print(rgb)

    obuff=cv2.resize(rgb, (64, 64))#, interpolation = cv2.INTER_NEAREST)
    ret, obuff = cv2.threshold(obuff, 128, 255, cv2.THRESH_BINARY)
    #print(obuff)
    #break
    
    nobuff = bytearray(int(64*64/8))  # bitmap picture, 1 Byte = 8 pixels
    cbit = 0
    for i in range(64*64):
        bout = obuff[i//64][i%64] > 0
        if bout:
            nobuff[i//8] |= 1 << (i%8)
            
    print(nobuff)

    sbuff = bytearray(65)
    off = 0
    for r in range(8):
        sbuff[0] = r
        for i in range(64):
            sbuff[1+i] = nobuff[off+i]
        off += 64

        ln = arduino.write(sbuff)
        print('%s' % ln)
        time.sleep(0.1)
   
    cv2.imshow('frame', obuff)
    cv2.waitKey(2)
    #time.sleep(2)
    #break


bytearray(b'\xc0\xff\xff\xff\xff\xff\xd1\xff\xc0\xff\xff\xff\xff\xff\xf3\xff\x80\xff\xff\xff\xff\xff\xf3\xff\x00\xff\xfb\xff\xff\xff\xf3\xff\x00\xfe\xfb\xff\xff\xff\xd1\xff\x00\xfc\xf7\xff\xff\xff\xd1\xff\x00\xf8\xff\xff\xff\xff\xc0\xff\x00\xc0\xff\xff\xff\x7f\xc0\xff\x00\xc0\xff\xff\xff\x7f\xe0\xff\x00\x80\xff\xff\xff?\xe0\xff\x00\x00\xfe\xff\xff\x1f\xe0\xff\x00\x00\xf7\x1f\xf8\x0f\xe0\xff\x00\x00\xf4\x0f\xf0\x0f\xe0\xff\x00\x00\xf0\x07\xf0\x03\xe0\xff\x00\x00\xfe\x03\xf0\x01\xe0\xff\x00\x00\xfc!\xf0\x01\xe0\xff\x00\x00\xbc\xf0\xf3\x00\xe0\xff\x00\x00\xfc\xf0\xc7\x00\xe0\xff\x00\x00\\\xf0\x0f\x00\xe0\xff\x00\x00\x0c\xf0\x0f\x00\xe0\xff\x00\x00\x00\xe0\x0f\x00\xe0\xff\x00\x00\x00\xe0\x1f\x00\xc0\xff\x00\x00\x00\xc0\x9f\x07\xc0\xff\x00\x00\x00\xe0\xdf\x03\xc0\xff\x00\x00\x00\xe0\x7f\x00\xe0\xff\x00\x00\x00\xf0?\x00\xe0\xff\x00\x00\x00\xc0?\x00\xe0\xff\x00\x00\x00\x00?\x00\xc0\xff\x00\x00 \x00?\x00\xc0\xff\x00\x00\x00\x001\x00\xc0\xff\x00\x00\x00\x001\x00\x80\xff\x00\x00\x00\x001\x00\x80

KeyboardInterrupt: 

In [ ]:
cv2.destroyAllWindows()

In [ ]:
ret, frame = cap.read()
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
print(rgb[0][0])

In [ ]:
#cap.release()
import random

In [ ]:
while(True):
#for i in range(2):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    W = 200
    roff = random.randrange((720-W)*(720-W))
    for r in range(W):
        for q in range(W):
            rgb[(roff//720)+r][(roff%720)+q] = [0x0, 0xff, 0x00, 0xff]
    
    ob = cv2.resize(rgb, (64,64))
    cv2.imshow('image', random.randrange(3,10)*rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()
   


In [ ]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [ ]:
buff=bytearray(65)

In [ ]:
arduino.write(b'\x04'+buff)

In [ ]:
for i in range(65):
    buff[i] = i

In [ ]:
for j in range(10):
    for i in range(10):
        buff[0] = i
        ln = arduino.write(buff)
        print('%s' % ln)
        time.sleep(0.1)


In [ ]:
while True:
    print(arduino.readline())